In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File"  # file without extention
sheets = "85 Sentence"  # 2 gram target, 2 gram hybrid
file_ext = "85 Sentence"
first_column = 2  # start column location number
end_column = 3

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

In [4]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [6]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [11]:
df_file = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name=f"{sheets}")
#df_file.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=True)
df_file.reset_index(drop=True, inplace=True)
df_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
3,çok iyi değil mi tam böyle gerçek gibi gerçek ...,3,6,çok iyi değil mi tam böyle gerçek gibi gerçek ...,OmT_BtZh8qU,https://www.youtube.com/watch?v=OmT_BtZh8qU&t=3s,"[9, 28, 14, 5, 112, 56, 183, 27, 183, 27, 60]"
4,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[39, 0, 181, 40, 3, 34, 6, 181, 40]"
...,...,...,...,...,...,...,...
80,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
81,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
82,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
83,tamam peki pekala pekala tamam,1408,1410,tamam peki pekala pekala tamam,d9tZZGbLQMU,https://www.youtube.com/watch?v=d9tZZGbLQMU&t=...,NaN


In [18]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 1]
check_list = df_check["word"].to_list()
remove_list = []
df_file_var = df_file
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file_var = df_file_var[df_file_var["search_string"] != f"{var_sent}"]
        df_file_var_count = word_count_result(df_file_var, df_file_var.columns[0:1])
        df_check = df_file_var_count[df_file_var_count["word_count"] > 1]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        pass

df_file_var.reset_index(drop=True, inplace=True)
df_file_var

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
1,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...,"[112, 71, 44, 172]"
2,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
3,diye söyle bak hala,104,106,o taşlarını gecesi zor için yoğurduktan sonra ...,P5bYtFrqw68,https://www.youtube.com/watch?v=P5bYtFrqw68&t=...,"[123, 161, 64, 167]"
4,içinde tam işte uzun,303,305,çıkıyor içinde tam işte uzun çıkmıyor para çık...,n5b8T70avq8,https://www.youtube.com/watch?v=n5b8T70avq8&t=...,"[158, 112, 108, 172]"
...,...,...,...,...,...,...,...
57,bile yok bu da benim için aslında önemli bir ş...,131,134,bulabilirsiniz şu an görünce bile yok bu da be...,aqQ7IUho9pA,https://www.youtube.com/watch?v=aqQ7IUho9pA&t=...,"[59, 22, 1, 15, 31, 4, 174, 156, 0, 16, 68]"
58,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[39, 0, 181, 40, 3, 34, 6, 181, 40]"
59,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
60,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"


In [19]:
remove_list

['şu adam şu adam',
 'ol başka biri ol',
 'olan oldu artık geri',
 'tabii onlar tabii ona',
 'biri büyük biri küçük',
 'bize lazım olacak çünkü',
 'nerede bizim gibi nasıl',
 'olsun ki sadece biz bize',
 'geldi ona kadar ona kadar',
 'size söyle merhaba merhaba',
 'onları biliyor musun çünkü',
 'ile hemen hemen aynı hiçbir',
 'tamam\xa0peki\xa0pekala\xa0pekala\xa0tamam',
 'aslında seninle birlikte orada',
 'hemen ve her gün her dakika oluyor',
 'kız seni gel buraya gel buraya kız gel',
 'biraz daha devam et devam et tamam oldu',
 'bence çok güzel bence çok güzel senin zaten',
 'burada böyle hayır öyle bir şey olabilir mi asla asla',
 'çok iyi değil mi tam böyle gerçek gibi gerçek gibi yani']

In [20]:
word_count_result(df_file_var, df_file.columns[0:1])

,word,word_count
0,bir,14
1,çok,11
2,için,7
3,değil,7
4,güzel,6
...,...,...
186,hemen,1
187,hayır,1
188,haydi,1
189,harika,1


In [ ]:
df_file_var.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx", index=False)

In [10]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])
df_file_count.sort_values(by="word_count", ascending=False)
df_file_count

,word,word_count
0,bir,15
1,çok,14
2,güzel,8
3,bakalım,8
4,değil,8
...,...,...
195,ister,1
196,hiç,1
197,hey,1
198,herkes,1


In [11]:
df_check = df_file_count[df_file_count["word_count"] > 2]
df_check

,word,word_count
0,bir,15
1,çok,14
2,güzel,8
3,bakalım,8
4,değil,8
...,...,...
79,diye,3
80,sen,3
81,şu,3
82,artık,3


In [12]:
check_list = df_check["word"].to_list()
check_list

['bir',
 'çok',
 'güzel',
 'bakalım',
 'değil',
 'için',
 'mi',
 'çünkü',
 'tamam',
 'ne',
 'kadar',
 'buraya',
 'böyle',
 'yani',
 'şey',
 'bu',
 'olsun',
 've',
 'oluyor',
 'var',
 'biraz',
 'şimdi',
 'daha',
 'gerçekten',
 'önce',
 'musun',
 'gibi',
 'geliyor',
 'gel',
 'ya',
 'zaman',
 'en',
 'ona',
 'hemen',
 'şekilde',
 'peki',
 'tam',
 'ben',
 'tekrar',
 'ama',
 'yok',
 'her',
 'aynı',
 'birlikte',
 'biri',
 'hiçbir',
 'küçük',
 'söyle',
 'tabii',
 'o',
 'kız',
 'uzun',
 'neler',
 'göre',
 'neden',
 'olacak',
 'nasıl',
 'mı',
 'lütfen',
 'zaten',
 'buna',
 'de',
 'bana',
 'orada',
 'benim',
 'senin',
 'biliyor',
 'biz',
 'bize',
 'bizim',
 'bunu',
 'öyle',
 'da',
 'son',
 'önemli',
 'olduğunu',
 'aslında',
 'olarak',
 'ki',
 'diye',
 'sen',
 'şu',
 'artık',
 'oldu']

In [13]:
len(check_list)

84

In [14]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 2]
check_list = df_check["word"].to_list()
remove_list = []
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file = df_file[df_file["search_string"] != f"{var_sent}"]
        df_file_count = word_count_result(df_file, df_file.columns[0:1])
        df_check = df_file_count[df_file_count["word_count"] > 2]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        df_file = df_file

In [15]:
df_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
1,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
2,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
3,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...,"[112, 71, 44, 172]"
4,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
...,...,...,...,...,...,...,...
80,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[39, 0, 181, 40, 3, 34, 6, 181, 40]"
81,çok iyi değil mi tam böyle gerçek gibi gerçek ...,3,6,çok iyi değil mi tam böyle gerçek gibi gerçek ...,OmT_BtZh8qU,https://www.youtube.com/watch?v=OmT_BtZh8qU&t=3s,"[9, 28, 14, 5, 112, 56, 183, 27, 183, 27, 60]"
82,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
83,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"


In [16]:
remove_list

['çünkü hiçbir zaman bana göre',
 'bakalım nasıl olacak ve buna',
 'bakalım son olarak neler var neler']

In [17]:
word_count_result(df_file, df_file.columns[0:1])

,word,word_count
0,bir,15
1,çok,14
2,güzel,8
3,değil,8
4,tamam,7
...,...,...
195,ister,1
196,hiç,1
197,hey,1
198,herkes,1


In [ ]:
test = ["aet","önce"]

In [ ]:
test in check_list

In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 2]
check_list = df_check["word"].to_list()
i = 0
while i < len(df_file):
    var_sent in df_file.loc[i,"search_string"]
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file = df_file[df_file["search_string"] != f"{var_sent}"]
        df_file_count = word_count_result(df_file, df_file.columns[0:1])
        df_check = df_file_count[df_file_count["word_count"] > 1]
        check_list = df_check["word"].to_list()
    else:
        df_file = df_file
    i += 1

In [ ]:
df_file

In [ ]:
check_list

In [ ]:
len(check_list)

In [ ]:
while var_sent in df_file["search_string"]:
    print(var_sent)

In [ ]:
all_list = []
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    all_list.append(result_list)

In [ ]:
all_list

In [ ]:
test_list = []
for j in all_list:
    if all(j):
        test_list.append(j)
    else:
        pass

In [ ]:
test_list

In [ ]:
result_list

In [ ]:
all(result_list)

In [ ]:
a = []

In [ ]:
a.append(True)

In [ ]:
a.append(False)

In [ ]:
a